In [1]:
# Cell 1: Update System and CUDA Setup
# Update system and CUDA setup.
!sudo pacman -Syu cuda libpng sdl2 sdl2_image sdl2_mixer sdl2_ttf libjpeg libtiff alsa-lib --noconfirm

Place your right index finger on the fingerprint reader
Failed to match fingerprint
Place your right index finger on the fingerprint reader
:: Synchronizing package databases...
 core
 extra
 community
 multilib
 chaotic-aur
 blackarch
otic-aur           670.9 KiB   769 KiB/s 00:01 [----------------------] 100%;37m o  o  ]  64%r            10.7 KiB  0.00   B/s --:-- [co  o  o  o  o  o  o  ]   1%warning: cuda-12.6.2-2 is up to date -- reinstalling
:: Starting full system upgrade...
resolving dependencies...
looking for conflicting packages...

Package (9)          Old Version  New Version  Net Change

extra/alsa-lib       1.2.12-1     1.2.12-1       0.00 MiB
extra/cuda           12.6.2-2     12.6.2-2       0.00 MiB
extra/libjpeg-turbo  3.0.4-1      3.0.4-1        0.00 MiB
extra/libpng         1.6.44-1     1.6.44-1       0.00 MiB
extra/libtiff        4.7.0-1      4.7.0-1        0.00 MiB
extra/sdl2           2.30.8-1     2.30.8-1       0.00 MiB
extra/sdl2_image     2.8.2-6      2.8.2-6   

In [ ]:
# Check our cuda setup
!nvidia-smi

In [ ]:
# Cell 2: Install Required Packages
# Install necessary packages
!pip install --no-binary :all: pygame --break-system-packages
!pip show pygame
!pip install tensorrt numpy -U --index-url https://download.pytorch.org/whl/cu124 --break-system-packages

In [ ]:
# Cell 3: Import Libraries
import math
import random
from typing import List, Tuple
import pygame
from pygame import mixer

import os
os.environ["SDL_VIDEODRIVER"] = "x11"

In [ ]:
# Cell 4: Initialize Pygame
# Initialize pygame
if not pygame.get_init():
    pygame.init()
    print("Current SDL video driver:", pygame.display.get_driver())

In [ ]:
# Cell 5 Create screen
if 'screen' not in globals():
    screen = pygame.display.set_mode((800, 600))
    pygame.event.pump()  # Ensure that Pygame processes events after creating the screen

In [ ]:
# Cell 6: Load Assets
# Background
if 'background' not in globals():
    background = pygame.image.load('background.png')

In [ ]:
# Sound
mixer.music.load("background.wav")
mixer.music.play(-1)

In [ ]:
# Caption and Icon
pygame.display.set_caption("Space Invader")
icon = pygame.image.load('ufo.png')
pygame.display.set_icon(icon)

In [ ]:
# Cell 7: Load Player
# Player setup
if 'playerImg' not in globals():
    playerImg = pygame.image.load('player.png')
    playerX = 370
    playerY = 480
    playerX_change = 0

In [ ]:
# Cell 8: Load Enemies
# Enemy setup
if not enemyImg:
    num_of_enemies = 6
    for i in range(num_of_enemies):
        enemyImg.append(pygame.image.load('enemy.png'))
        enemyX.append(random.randint(0, 736))
        enemyY.append(random.randint(50, 150))
        enemyX_change.append(4)
        enemyY_change.append(40)
# Cell 9: Bullet Setup
# Bullet setup
bulletImg = pygame.image.load('bullet.png')
bulletX = 0
bulletY = 480
bulletX_change = 0
bulletY_change = 10
bullet_state = "ready"

In [ ]:
# Cell 10: Score Setup
# Score setup
score_value = 0
font = pygame.font.Font('freesansbold.ttf', 32)

In [ ]:
textX = 10
testY = 10

In [ ]:
# Game Over setup
over_font = pygame.font.Font('freesansbold.ttf', 64)

In [ ]:
# Cell 11: Functions
# Define necessary functions
def show_score(x, y):
    score = font.render("Score : " + str(score_value), True, (255, 255, 255))
    screen.blit(score, (x, y))

In [ ]:
def game_over_text():
    over_text = over_font.render("GAME OVER", True, (255, 255, 255))
    screen.blit(over_text, (200, 250))

In [ ]:
def player(x, y):
    screen.blit(playerImg, (x, y))

In [ ]:
def enemy(x, y, i):
    screen.blit(enemyImg[i], (x, y))

In [ ]:
def fire_bullet(x, y):
    global bullet_state
    bullet_state = "fire"
    screen.blit(bulletImg, (x + 16, y + 10))

In [ ]:
def isCollision(enemyX, enemyY, bulletX, bulletY):
    distance = math.sqrt(math.pow(enemyX - bulletX, 2) + (math.pow(enemyY - bulletY, 2)))
    if distance < 27:
        return True
    else:
        return False

In [ ]:
# Cell 12: Main Game Loop
# Game Loop
running = True
while running:
    # RGB = Red, Green, Blue
    screen.fill((0, 0, 0))
    # Background Image
    screen.blit(background, (0, 0))
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        # If keystroke is pressed check whether it's right or left
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_LEFT:
                playerX_change = -5
            if event.key == pygame.K_RIGHT:
                playerX_change = 5
            if event.key == pygame.K_SPACE:
                if bullet_state == "ready":
                    bulletSound = mixer.Sound("laser.wav")
                    bulletSound.play()
                    # Get the current x coordinate of the spaceship
                    bulletX = playerX
                    fire_bullet(bulletX, bulletY)

        if event.type == pygame.KEYUP:
            if event.key == pygame.K_LEFT or event.key == pygame.K_RIGHT:
                playerX_change = 0

    # Player Movement
    playerX += playerX_change
    if playerX <= 0:
        playerX = 0
    elif playerX >= 736:
        playerX = 736

    # Enemy Movement
    for i in range(num_of_enemies):

        # Game Over
        if enemyY[i] > 440:
            for j in range(num_of_enemies):
                enemyY[j] = 2000
            game_over_text()
            break

        enemyX[i] += enemyX_change[i]
        if enemyX[i] <= 0:
            enemyX_change[i] = 4
            enemyY[i] += enemyY_change[i]
        elif enemyX[i] >= 736:
            enemyX_change[i] = -4
            enemyY[i] += enemyY_change[i]

        # Collision
        collision = isCollision(enemyX[i], enemyY[i], bulletX, bulletY)
        if collision:
            explosionSound = mixer.Sound("explosion.wav")
            explosionSound.play()
            bulletY = 480
            bullet_state = "ready"
            score_value += 1
            enemyX[i] = random.randint(0, 736)
            enemyY[i] = random.randint(50, 150)

        enemy(enemyX[i], enemyY[i], i)

    # Bullet Movement
    if bulletY <= 0:
        bulletY = 480
        bullet_state = "ready"

    if bullet_state == "fire":
        fire_bullet(bulletX, bulletY)
        bulletY -= bulletY_change

    player(playerX, playerY)
    show_score(textX, testY)
    pygame.display.update()